<font size=5><b>Download and Clean Dataset <font size/></b>

Let's start by importing the pandas and the Numpy libraries.

In [2]:
import pandas as pd
import numpy as np


We will be using the dataset provided in the assignment

<font size=3><b>
    
The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement

2. Blast Furnace Slag

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

<font size/><b/>

Let's read the dataset into a pandas dataframe.

In [5]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


<font size2><b>
So the first concrete sample has 

540 cubic meter of cement, 
    
    
0 cubic meter of blast furnace slag, 
    
0 cubic meter of fly ash, 
    
162 cubic meter of water, 
    
2.5 cubic meter of superplaticizer, 
    
1040 cubic meter of coarse aggregate,
    
676 cubic meter of fine aggregate. 

Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.

<font size/><b>

<b>Let's check how many data points we have <b/>

In [6]:
concrete_data.shape

(1030, 9)


So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [7]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [8]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

<b>Split data into predictors and target <b/>

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns


In [9]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [10]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [11]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [12]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

<font size=4><b>Import Keras<font size/><b/>

<font size=2><b>Let's go ahead and import the Keras library
<font size/><b/>


In [13]:
import keras

Using TensorFlow backend.


As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [14]:
from keras.models import Sequential
from keras.layers import Dense

In [15]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

Let's import scikit-learn in order to randomly split the data into a training and test sets

In [16]:
from sklearn.model_selection import train_test_split

Splitting the data into a training and test sets by holding 30% of the data for testing

In [17]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

<font size=3><b>Train and Test the Network <font size/><b/>


Let's call the function now to create our model.

In [19]:
# build the model
model = regression_model()

Instructions for updating:
Colocations handled automatically by placer.


Next, we will train the model for 50 epochs.

In [20]:
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
721/721 [==============================] - 1s 942us/step - loss: 576570.2505
Epoch 2/50
721/721 [==============================] - 0s 150us/step - loss: 314573.5016
Epoch 3/50
721/721 [==============================] - 0s 124us/step - loss: 145771.9020
Epoch 4/50
721/721 [==============================] - 0s 156us/step - loss: 64516.4807
Epoch 5/50
721/721 [==============================] - 0s 138us/step - loss: 33715.6068
Epoch 6/50
721/721 [==============================] - 0s 137us/step - loss: 23235.3935
Epoch 7/50
721/721 [==============================] - 0s 140us/step - loss: 20162.1198
Epoch 8/50
721/721 [==============================] - 0s 137us/step - loss: 18837.3822
Epoch 9/50
721/721 [==============================] - 0s 137us/step - loss: 17854.6543
Epoch 10/50
721/721 [==============================] - 0s 140us/step - loss: 16905.0794
Epoch 11/50
721/721 [==============================] - 0s 135us/step - loss: 1

Next we need to evaluate the model on the test data.

In [21]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 239us/step


2356.134601284385

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.

In [22]:
from sklearn.metrics import mean_squared_error

In [23]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

2356.1346326894727 0.0


Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [24]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 366.8025270233648
MSE 2: 147.8319030564194
MSE 3: 82.17566142653185
MSE 4: 68.86940603657447
MSE 5: 57.157245870547
MSE 6: 57.69126378525422
MSE 7: 58.860775746959696
MSE 8: 46.804364127248625
MSE 9: 50.72255724070527
MSE 10: 52.498615919193405
MSE 11: 58.896903942318026
MSE 12: 46.18655167119788
MSE 13: 56.63879850690033
MSE 14: 53.53421337781987
MSE 15: 51.49435485296651
MSE 16: 57.25458656236963
MSE 17: 70.07659196082056
MSE 18: 50.77383524083011
MSE 19: 45.51473137321596
MSE 20: 59.784644086769866
MSE 21: 59.405603847071575
MSE 22: 44.839949709697834
MSE 23: 45.295604631738755
MSE 24: 51.54524175403188
MSE 25: 57.9223351463145
MSE 26: 50.567638236727916
MSE 27: 65.75501224061047
MSE 28: 45.94555985040263
MSE 29: 52.33682549732789
MSE 30: 50.366716489822736
MSE 31: 50.04296710807529
MSE 32: 43.13048942343703
MSE 33: 50.73487461423411
MSE 34: 52.465728389406664
MSE 35: 47.07274575835293
MSE 36: 52.99485615850652
MSE 37: 52.698053409366544
MSE 38: 52.33647187859495
MSE 39: 60.6